In [1]:
from scipy import signal as sig

# imports from custom library
import sys
sys.path.append('../../')
import matplotlib.pyplot as plt
plt.rc('text', usetex=True)
from mlrefined_libraries import convnets_library as convlib
from mlrefined_libraries import basics_library as baslib
from mlrefined_libraries import superlearn_library as superlearn
from mlrefined_libraries import multilayer_perceptron_library as network_lib

import autograd.numpy as np
from autograd import grad as compute_grad   

import autograd.numpy as np
import numpy as npo

import pandas as pd
import csv
import pickle
import glob
import time
import copy
from datetime import datetime 

#this is needed to compensate for matplotlib notebook's tendancy to blow up images when plotted inline
%matplotlib notebook
from matplotlib import rcParams
rcParams['figure.autolayout'] = True

%load_ext autoreload
%autoreload 2

## Baseline convolution function

In [2]:
import scipy.signal as signal

def ScipyConv(image, kernel):
    
    # flip kernel
    kernel = np.flipud(np.fliplr(kernel))
    
    # compute convolution
    conv = signal.convolve2d(image, kernel, boundary='fill', fillvalue=0, mode='same')
    return conv  

# Concatenated image version

In [3]:
# sliding window for image augmentation
def sliding_window_image(image, kernel, stride):
    windowed_image = []
    for i in np.arange(0, np.shape(image)[0]-kernel.shape[0]+1, stride):
        for j in np.arange(0, np.shape(image)[1]-kernel.shape[1]+1, stride):
             windowed_image.append(image[i:i+kernel.shape[0], j:j+kernel.shape[1]].flatten())
            
    return np.array(windowed_image)

# pad image with appropriate number of zeros for convolution
def pad_image(image,kernel_size):
    odd_nums = np.array([int(2*n + 1) for n in range(100)])
    val = kernel_size[0]
    pad_val = np.argwhere(odd_nums == val)[0][0]
    image_padded = np.zeros((np.shape(image) + 2*pad_val))
    image_padded[pad_val:-pad_val,pad_val:-pad_val] = image
    return image_padded          

In [4]:
# generate test image and kernel
image = np.random.randn(5,5)
kernel = np.ones((3,3))
kernel_size = kernel.shape

In [5]:
# pad image
padded_image = pad_image(image,kernel_size)

# window image
wind_img = sliding_window_image(padded_image,kernel,stride = 1)

# produce matrix multiplication convolution 
conv2 = np.dot(wind_img,kernel.flatten()[:,np.newaxis])

# reshape convolution into array
conv2.shape = (np.shape(image))

In [6]:
print(np.shape(image))
print(np.shape(padded_image))
print(np.shape(wind_img))
print(np.shape(conv2))

(5, 5)
(7, 7)
(25, 9)
(5, 5)


In [7]:
#### baseline convolution #####
conv1 = ScipyConv(image,kernel)

In [8]:
#### compare convolutions ####
error = np.linalg.norm(conv1 - conv2)
print (error)
print (np.shape(conv1),np.shape(conv2))

1.62908915145e-15
(5, 5) (5, 5)


In [9]:
# activation 
def activation(t):
    return np.maximum(0,t)

# output of activation
a_conv = activation(conv2)

In [10]:
# pool operation
kernel2 = np.ones((2,2))
wind_conv = sliding_window_image(a_conv,kernel2,stride = 2)

# max pooling
max_pool = np.max(wind_conv,axis = 1)

In [11]:
print (np.shape(wind_conv))
print (np.shape(max_pool))

(4, 4)
(4,)


In [12]:
# back to an image
max_pool.shape = (int(np.size(max_pool)**(0.5)),int(np.size(max_pool)**(0.5)))

In [13]:
np.shape(max_pool)

(2, 2)

# Tensor version

In [14]:
# sliding window for image augmentation
def sliding_window_tensor(tensor, kernel, stride):
    windowed_tensor = []
    for i in np.arange(0, np.shape(tensor)[1]-kernel.shape[0]+1, stride):
        for j in np.arange(0, np.shape(tensor)[2]-kernel.shape[1]+1, stride):
            sock = copy.deepcopy(tensor[:,i:i+kernel.shape[0], j:j+kernel.shape[1]])
            windowed_tensor.append(sock)
    
    # re-shape properly
    windowed_tensor = np.array(windowed_tensor)
    windowed_tensor = windowed_tensor.swapaxes(0,1)    
    windowed_tensor = np.reshape(windowed_tensor,(np.shape(windowed_tensor)[0]*np.shape(windowed_tensor)[1],np.shape(windowed_tensor)[2]*np.shape(windowed_tensor)[3]))   
    return windowed_tensor


# pad image with appropriate number of zeros for convolution
def pad_tensor(tensor,kernel):
    odd_nums = np.array([int(2*n + 1) for n in range(100)])
    val = kernel.shape[0]
    pad_val = np.argwhere(odd_nums == val)[0][0]
    tensor_padded = np.zeros((np.shape(tensor)[0], np.shape(tensor)[1] + 2*pad_val,np.shape(tensor)[2] + 2*pad_val))
    tensor_padded[:,pad_val:-pad_val,pad_val:-pad_val] = tensor
    return tensor_padded    

In [15]:
# create test tensor
tensor = np.random.randn(3,5,5)
kernel = np.ones((3,3))

In [16]:
# pad tensor
padded_tensor = pad_tensor(tensor,kernel)

# window tensor
wind_tensor = sliding_window_tensor(padded_tensor,kernel,stride = 1)

# # produce matrix multiplication convolution 
conv2 = np.dot(wind_tensor,kernel.flatten()[:,np.newaxis])

# # reshape convolution into array
conv2.shape = (np.shape(tensor))

In [17]:
print(np.shape(tensor))
print(np.shape(padded_tensor))
print(np.shape(wind_tensor))
print(np.shape(conv2))

(3, 5, 5)
(3, 7, 7)
(75, 9)
(3, 5, 5)


In [18]:
#### baseline convolution #####
convs = []
for i in range(np.shape(tensor)[0]):
    conv1 = ScipyConv(tensor[i,:,:],kernel)
    convs.append(conv1)
convs = np.asarray(convs)

In [19]:
#### compare convolutions ####
error = np.linalg.norm(convs - conv2)
print (error)
print (np.shape(convs),np.shape(conv2))

2.97635884971e-15
(3, 5, 5) (3, 5, 5)


In [20]:
# output of activation
a_conv = activation(conv2)

In [21]:
# pool operation
kernel2 = np.ones((2,2))
wind_conv = sliding_window_tensor(a_conv,kernel2,stride = 2)

# max pooling
max_pool = np.max(wind_conv,axis = 1)

In [22]:
print (np.shape(wind_conv))
print (np.shape(max_pool))

(12, 4)
(12,)


In [23]:
# reshape into tensor
max_pool.shape = (np.shape(tensor)[0],int(np.shape(wind_conv)[1]**(0.5)),int(np.shape(wind_conv)[1]**(0.5)))

In [24]:
np.shape(max_pool)

(3, 2, 2)

# Test with tensor and multiple kernels

In [25]:
# create test tensor
tensor = np.random.randn(3,28,28)
kernels = np.random.randn(8,3,3)
kernel = kernels[0]

# pad tensor
padded_tensor = pad_tensor(tensor,kernel)

# window tensor
wind_tensor = sliding_window_tensor(padded_tensor,kernel,stride = 1)

In [26]:
#### baseline convolution #####
convs = []
for kernel in kernels:
    temp = []
    for i in range(np.shape(tensor)[0]):
        conv1 = ScipyConv(tensor[i,:,:],kernel)
        temp.append(conv1)
    temp = np.asarray(temp)
    convs.append(temp)
convs = np.asarray(convs)

##### use tensor calculation from above ######

startTime= datetime.now() 

conv2 = []
padded_tensor = pad_tensor(tensor,kernel)
wind_tensor = sliding_window_tensor(padded_tensor,kernel,stride = 1)
for kernel in kernels:
    # # produce matrix multiplication convolution 
    conv = np.dot(wind_tensor,kernel.flatten()[:,np.newaxis])

    # # reshape convolution into array
    conv.shape = (np.shape(tensor))
    conv = np.asarray(conv)
    conv2.append(conv)
conv2 = np.asarray(conv2)

timeElapsed=datetime.now()-startTime 

print('Time elpased (hh:mm:ss.ms) {}'.format(timeElapsed))

Time elpased (hh:mm:ss.ms) 0:00:00.010831


In [27]:
# # SUPER COMPACT VERSION
# # produce matrix multiplication convolution 
num_kernels = np.shape(kernels)[0]

startTime= datetime.now() 

conv3 = np.dot(wind_tensor,kernels.reshape(np.shape(kernels)[0],np.shape(kernels)[1]*np.shape(kernels)[2]).T).T

# # reshape convolution into array
a = np.shape(tensor)
a = list(a)
a.insert(0,num_kernels)
a = tuple(a)
conv3.shape = a

timeElapsed=datetime.now()-startTime 

print('Time elpased (hh:mm:ss.ms) {}'.format(timeElapsed))

Time elpased (hh:mm:ss.ms) 0:00:00.000367


In [28]:
#### compare convolutions  - either scipy or one kernel at a time ####
error = np.linalg.norm(conv2 - convs)
print (error)
print (np.shape(convs),np.shape(conv3))

6.19917216219e-14
(8, 3, 28, 28) (8, 3, 28, 28)


At the end of convolution we have `num_kernels` number of feature maps for our input data.

In [29]:
np.shape(conv2)

(8, 3, 28, 28)

Now to shove through activation and pool.

In [30]:
# output of activation
transformed_feature_maps = activation(conv2)

In [31]:
np.shape(transformed_feature_maps)

(8, 3, 28, 28)

In [32]:
# pool operation
kernel2 = np.ones((6,6))
stride = 3
new_tensors = []
for feature_map in transformed_feature_maps:
    # move over feature map and gather patches
    wind_conv = sliding_window_tensor(feature_map,kernel2,stride = stride)
    
    # max pool on each collected patch
    max_pool = np.max(wind_conv,axis = 1)
    
    # reshape into new tensor
    max_pool.shape = (np.shape(tensor)[0],int((np.shape(max_pool)[0]/float(np.shape(tensor)[0]))**(0.5)),int((np.shape(max_pool)[0]/float(np.shape(tensor)[0]))**(0.5)))
    
    # reshape into new downsampled pooled feature map
    new_tensors.append(max_pool)
    
# turn into array
new_tensors = np.asarray(new_tensors)

In [33]:
np.shape(new_tensors)

(8, 3, 8, 8)

Final step - as we feed into fully connected network component, make sure everything is reshaped correctly.  What the final output shape should be - one long vectorized sequence of 'feature maps' for each input image.  So here we are

In [34]:
# reshape into final feature vector to touch fully connected layer(s)
new_tensors = new_tensors.swapaxes(0,1)
new_tensors = new_tensors.reshape(np.shape(new_tensors)[0],np.shape(new_tensors)[1]*np.shape(new_tensors)[2]*np.shape(new_tensors)[3]).T
print (np.shape(new_tensors))

(512, 3)


All together we have.

In [35]:
# create test tensor
tensor = np.random.randn(3,28,28)
kernels = np.random.randn(8,3,3)
kernel = kernels[0]

# pad tensor
padded_tensor = pad_tensor(tensor,kernel)

# window tensor
wind_tensor = sliding_window_tensor(padded_tensor,kernel,stride = 1)

#### create convolution feature maps ####
feature_maps = np.dot(wind_tensor,kernels.reshape(np.shape(kernels)[0],np.shape(kernels)[1]*np.shape(kernels)[2]).T).T

# reshape feature maps back into arrays
shapes = np.shape(tensor)
shapes = list(shapes)
shapes.insert(0,num_kernels)
shapes = tuple(shapes)
feature_maps.shape = shapes

# push feature maps through activation
transformed_feature_maps = activation(feature_maps)

#### downsample via pooling ####
kernel2 = np.ones((6,6))
stride = 3
new_tensors = []
for feature_map in transformed_feature_maps:
    # move over feature map and gather patches
    wind_conv = sliding_window_tensor(feature_map,kernel2,stride = stride)
    
    # max pool on each collected patch
    max_pool = np.max(wind_conv,axis = 1)
    
    # reshape into new tensor
    max_pool.shape = (np.shape(tensor)[0],int((np.shape(max_pool)[0]/float(np.shape(tensor)[0]))**(0.5)),int((np.shape(max_pool)[0]/float(np.shape(tensor)[0]))**(0.5)))
    
    # reshape into new downsampled pooled feature map
    new_tensors.append(max_pool)
    
# turn into array
new_tensors = np.asarray(new_tensors)

# reshape into final feature vector to touch fully connected layer(s)
new_tensors = new_tensors.swapaxes(0,1)
new_tensors = new_tensors.reshape(np.shape(new_tensors)[0],np.shape(new_tensors)[1]*np.shape(new_tensors)[2]*np.shape(new_tensors)[3]).T
print (np.shape(new_tensors))

(512, 3)


# Test transformation on face images

Load in data, transform via original method, transform via new method, compare features to make sure everything looks good.

## Old way

In [36]:
 def load_data(datapath):
    # load in data
    data = np.loadtxt(datapath,delimiter = ',')

    # import data and reshape appropriately
    X = data[:,:-1]
    y = data[:,-1]
    y.shape = (len(y),1)
    
    X_square = np.zeros((len(y),28,28))
    for i in range(0,len(y)):
        X_square[i,:,:] = np.reshape(X[i,:],(28,28),1)
    
    return X,X_square,y

In [37]:
# load data
datapath = '../../mlrefined_datasets/convnet_datasets/feat_face_data.csv'
X,X_square, y = load_data(datapath)

In [38]:
X_square = X_square[:10]
X = X[:10,:]

In [39]:
# load kernels
kernels = convlib.image_viz.load_kernels()

# params
sliding_window_size = (6,6) 
stride=3
pooling_func= 'max'

# get number of images in the dataset
num_images = np.shape(X_square)[0]
        
# a test run to find the number of features with the params above
test = convlib.image_viz.make_feat(X_square[0,:,:], kernels, sliding_window_size=sliding_window_size, stride=stride)
num_features = np.shape(test)[0]

In [40]:
# start timer
startTime= datetime.now() 

# run old method
feat = []
for i in range(0,num_images):
    # extract features
    f = convlib.image_viz.make_feat(X_square[i,:,:], kernels, sliding_window_size=sliding_window_size,
                                            stride=stride, pooling_func=pooling_func) 
    # store it
    feat.append(f)
    
# convert to array
feat = np.asarray(feat)

# time for measurment
timeElapsed=datetime.now()-startTime 

print('Time elpased (hh:mm:ss.ms) {}'.format(timeElapsed))

Time elpased (hh:mm:ss.ms) 0:00:00.038509


## New way - non-compact image version

Re-make original exposing all looping structures etc.,

Far too slow - takes over a minute for just 4000 images - can't use.

In [41]:
# copy over test kernels
new_kernels = []
kernels = convlib.image_viz.load_kernels()
for ind, kernel in kernels.items():
    new_kernels.append(kernel)
new_kernels = np.asarray(new_kernels)
kernels = copy.deepcopy(new_kernels)

In [42]:
# assign tensor name
tensor = copy.deepcopy(X_square)

In [43]:
# start timer
startTime = datetime.now() 

# sliding window for image augmentation
def sliding_window_image(image, kernel_size, stride):
    windowed_image = []
    for i in np.arange(0, np.shape(image)[0]-kernel_size+1, stride):
        for j in np.arange(0, np.shape(image)[1]-kernel_size+1, stride):
             windowed_image.append(image[i:i+kernel_size, j:j+kernel_size].flatten())
            
    return np.array(windowed_image)

# pad image with appropriate number of zeros for convolution
def pad_image(image,kernel_size):
    odd_nums = np.array([int(2*n + 1) for n in range(100)])
    pad_val = np.argwhere(odd_nums == kernel_size)[0][0]
    image_padded = np.zeros((np.shape(image) + 2*pad_val))
    image_padded[pad_val:-pad_val,pad_val:-pad_val] = image
    return image_padded          

#### loop over each image, shove through filters and make feature maps, then downsample and pool
new_tensors = []
kernel_size = kernels[0].shape[0]
pool_kernel_size = 6
stride = 3

#### loop over images
for image in tensor:
    # pad image with zeros
    padded_image = pad_image(image,kernel_size)

    #### loop over kernels and construct feature map for each kernel
    downsampled_feature_maps = []
    for kernel in kernels:
        # window image
        wind_img = sliding_window_image(padded_image,kernel_size,stride = 1)
    
        # make convolution feature map - via matrix multiplication over windowed tensor 
        feature_map = np.dot(wind_img,kernel.flatten()[:,np.newaxis])
        
        # reshape convolution feature map into array
        feature_map = np.reshape(feature_map,(np.shape(image)))

        # now shove result through nonlinear activation
        feature_map = activation(feature_map)

        #### now pool / downsample feature map, first window then pool on each window
        wind_featmap = sliding_window_image(feature_map,pool_kernel_size,stride = stride)

        # max pool on each collected patch
        max_pool = np.max(wind_featmap,axis = 1)

        # reshape into new tensor
        max_pool = np.reshape(max_pool, (int((np.size(max_pool))**(0.5)),int((np.size(max_pool))**(0.5))))

        # reshape into new downsampled pooled feature map
        downsampled_feature_maps.append(max_pool)
        
    ## re-shape downsampled_feature_maps and store
    new_tensors.append(downsampled_feature_maps)

# reshape new tensor properly
new_tensors = np.reshape(new_tensors, (np.shape(new_tensors)[0],np.shape(new_tensors)[1],np.shape(new_tensors)[2]*np.shape(new_tensors)[3]))
new_tensors = np.reshape(new_tensors, (np.shape(new_tensors)[0],np.shape(new_tensors)[1]*np.shape(new_tensors)[2]),order = 'F')

# time for measurment
timeElapsed=datetime.now()-startTime 

print('Time elpased (hh:mm:ss.ms) {}'.format(timeElapsed))

Time elpased (hh:mm:ss.ms) 0:00:00.536617


In [44]:
np.linalg.norm(new_tensors - feat)

6.6432407931151067e-16

In [45]:
np.shape(new_tensors)

(10, 512)

## New way - non-compact tensor version

#### A somewhat effecient version of the feature transform code that is still somewhat understandable, about 4 times faster than most naive version.

Convert kernel dictionary to tensor.

In [46]:
# copy over test kernels
new_kernels = []
kernels = convlib.image_viz.load_kernels()
for ind, kernel in kernels.items():
    new_kernels.append(kernel)
new_kernels = np.asarray(new_kernels)
kernels = copy.deepcopy(new_kernels)

# assign tensor name
tensor = X_square

Now run.

In [47]:
X.shape

(10, 784)

In [48]:
# start timer
startTime= datetime.now() 

# activation 
def activation(t):
    return np.maximum(0,t)

# square up bro
tensor = np.reshape(X,(np.shape(X)[0],int((np.shape(X)[1])**(0.5)),int( (np.shape(X)[1])**(0.5))),order = 'F')

        
padded_tensor = pad_tensor(tensor,kernel)
        
# window tensor
wind_tensor = sliding_window_tensor(padded_tensor,kernel,stride = 1)

#### compute convolution feature maps / downsample via pooling one map at a time over entire tensor #####
kernel2 = np.ones((6,6))
stride = 3
new_tensors = []
for kernel in kernels:
    #### make convolution feature map - via matrix multiplication over windowed tensor 
    feature_map = np.dot(wind_tensor,kernel.flatten()[:,np.newaxis])

    # reshape convolution feature map into array
    feature_map = np.reshape(feature_map,np.shape(tensor))

    # now shove result through nonlinear activation
    feature_map = activation(feature_map)

    #### now pool / downsample feature map, first window then pool on each window
    wind_featmap = sliding_window_tensor(feature_map,kernel2,stride = stride)

    # max pool on each collected patch
    max_pool = np.max(wind_featmap,axis = 1)

    # reshape into new tensor
    max_pool = np.reshape(max_pool,(np.shape(tensor)[0],int((np.shape(max_pool)[0]/float(np.shape(tensor)[0]))**(0.5)),int((np.shape(max_pool)[0]/float(np.shape(tensor)[0]))**(0.5))))

    # reshape into new downsampled pooled feature map
    new_tensors.append(max_pool)

# turn into array
new_tensors = np.array(new_tensors)

# reshape into final feature vector to touch fully connected layer(s), otherwise keep as is in terms of shape
new_tensors = new_tensors.swapaxes(0,1)
new_tensors = np.reshape(new_tensors, (np.shape(new_tensors)[0],np.shape(new_tensors)[1],np.shape(new_tensors)[2]*np.shape(new_tensors)[3]))
new_tensors = np.reshape(new_tensors, (np.shape(new_tensors)[0],np.shape(new_tensors)[1]*np.shape(new_tensors)[2]),order = 'F')

# time for measurment
timeElapsed=datetime.now()-startTime 

print('Time elpased (hh:mm:ss.ms) {}'.format(timeElapsed))

Time elpased (hh:mm:ss.ms) 0:00:00.020514


In [49]:
np.linalg.norm(new_tensors - feat)

6.6432407931151067e-16

## New way - compact tensor version

#### More compact version - all kernel multiplications done at once - strangely is a bit slower than the one above, all the re-shaping must not be worth it, at least for this instance

In [50]:
# start timer
startTime= datetime.now() 

# pad tensor
kernel = kernels[0]
num_kernels = np.shape(kernels)[0]
padded_tensor = pad_tensor(tensor,kernel)

# window tensor
wind_tensor = sliding_window_tensor(padded_tensor,kernel,stride = 1)

#### create convolution feature maps ####
feature_maps = np.dot(wind_tensor,kernels.reshape(np.shape(kernels)[0],np.shape(kernels)[1]*np.shape(kernels)[2]).T).T

# reshape feature maps back into arrays
shapes = np.shape(tensor)
shapes = list(shapes)
shapes.insert(0,num_kernels)
shapes = tuple(shapes)
feature_maps.shape = shapes

# push feature maps through activation
# activation 
def activation(t):
    return np.maximum(0,t)

transformed_feature_maps = activation(feature_maps)

#### downsample via pooling ####
kernel2 = np.ones((6,6))
stride = 3
new_tensors = []
for feature_map in transformed_feature_maps:    
    # move over feature map and gather patches
    wind_featmap = sliding_window_tensor(feature_map,kernel2,stride = stride)
    
    # max pool on each collected patch
    max_pool = np.max(wind_featmap,axis = 1)

    # reshape into new tensor
    max_pool.shape = (np.shape(tensor)[0],int((np.shape(max_pool)[0]/float(np.shape(tensor)[0]))**(0.5)),int((np.shape(max_pool)[0]/float(np.shape(tensor)[0]))**(0.5)))

    # reshape into new downsampled pooled feature map
    new_tensors.append(max_pool)
    
# turn into array
new_tensors = np.asarray(new_tensors)

# reshape into final feature vector to touch fully connected layer(s), otherwise keep as is in terms of shape
new_tensors = new_tensors.swapaxes(0,1)
new_tensors = np.reshape(new_tensors, (np.shape(new_tensors)[0],np.shape(new_tensors)[1],np.shape(new_tensors)[2]*np.shape(new_tensors)[3]))
new_tensors = np.reshape(new_tensors, (np.shape(new_tensors)[0],np.shape(new_tensors)[1]*np.shape(new_tensors)[2]),order = 'F')

# time for measurment
timeElapsed=datetime.now()-startTime 

print('Time elpased (hh:mm:ss.ms) {}'.format(timeElapsed))

Time elpased (hh:mm:ss.ms) 0:00:00.022318


In [51]:
np.linalg.norm(new_tensors - feat)

7.7624475775262038e-16

# Really compact version - push all tensors together for compute

##### to do: probably not necessary for now, but intellectually appealing 

In [52]:
class naive_conv_layer:  
    '''
    A simple convnet module.  Here we calculate feature maps exactly one at a time, using
    a host of nested for-loops.  This means computation will be quite slow!  However this
    can still be used in theory as a fixed convolutional feature extractor or as a convolutional
    layer in a conv net (where the kernels are learned).
    '''   
    
    # a convolution function
    def conv_function(self,window):
        conv = np.sum(self.kernel*window)
        return conv

    # a pooling function
    def pool_function(self,window):
        pool = np.max(window)
        return pool

    # activation function
    def activation(self,window):
        a = np.maximum(0,window)
        return a

    # pad image with appropriate number of zeros for convolution
    def pad_image(self,image,kernel_size):
        odd_nums = np.array([int(2*n + 1) for n in range(100)])
        pad_val = np.argwhere(odd_nums == kernel_size)[0][0]
        image_padded = np.zeros((np.shape(image) + 2*pad_val))
        image_padded[pad_val:-pad_val,pad_val:-pad_val] = image
        return image_padded   

    # sliding window function, convolution or pooling done on each window
    def sliding_window_image(self,image,window_size,stride,func):
        # grab image size, set container for results
        image_size = np.shape(image)[0]
        results = []

        # slide window over input image with given window size / stride and function
        for i in np.arange(0, image_size - window_size + 1, stride):
            for j in np.arange(0, image_size - window_size + 1, stride):
                # now we have a window from our image, and use the desired 'func' to process it
                window = image[i:i+window_size,j:j+window_size]

                # process using input func
                processed_window = func(window)
                results.append(processed_window)

        # array-afy results
        results = np.array(results)

        # return results in numpy array format
        return results

    def make_feature_map(self,image,kernel):
        # square up input
        self.kernel = kernel
        img_size = int((np.size(image))**(0.5))
        image = np.reshape(image,(img_size,img_size))

        # pad image appropriately
        kernel_size = kernel.shape[0]
        padded_image = self.pad_image(image,kernel_size)

        # create feature map via convolution --> returns flattened convolution calculations
        conv_stride = 1
        feature_map = self.sliding_window_image(padded_image,kernel_size,conv_stride,self.conv_function)

        # reshape convolution feature map into array
        feature_map = np.reshape(feature_map,(np.shape(image)))

        # now shove result through nonlinear activation
        feature_map = self.activation(feature_map)

        #### now pool / downsample feature map, first window then pool on each window
        max_pool = self.sliding_window_image(feature_map,6,3,self.pool_function)

        # reshape into new tensor
        max_pool = np.reshape(max_pool, (int((np.size(max_pool))**(0.5)),int((np.size(max_pool))**(0.5))))

        return max_pool
    
    # convolution layer function - here we collect all of the feature maps and package them appropriately
    def conv_layer(self,tensor,kernels):   
        kernel = kernels[0]
        all_feature_maps = []
        for image in tensor:
            current_feat_maps = []
            for kernel in kernels:
                # compute feature map for current image using current convolution kernel
                feat_map = self.make_feature_map(image,kernel)

                # store feature maps of current kernel
                current_feat_maps.append(feat_map)

            # append all feature maps from current kernel to running list
            all_feature_maps.append(current_feat_maps)

        # convert to array and re-shape properly
        all_feature_maps = np.array(all_feature_maps)
        all_feature_maps = np.reshape(all_feature_maps,(np.shape(all_feature_maps)[0],np.prod(np.shape(all_feature_maps)[1:])),order = 'F')
        return all_feature_maps

In [53]:
class tensor_conv_layer:    
    # convolution function
    def conv_function(self,tensor_window):
        tensor_window = np.reshape(tensor_window,(np.shape(tensor_window)[0],np.shape(tensor_window)[1]*np.shape(tensor_window)[2]))
        t = np.dot(self.kernels,tensor_window.T)
        return t

    # pooling / downsampling parameters
    def pool_function(self,tensor_window):
        t = np.max(tensor_window,axis = (1,2))
        return t

    # activation 
    def activation(self,tensor_window):
        return np.maximum(0,tensor_window)

    # pad image with appropriate number of zeros for convolution
    def pad_tensor(self,tensor,kernel_size):
        odd_nums = np.array([int(2*n + 1) for n in range(100)])
        pad_val = np.argwhere(odd_nums == kernel_size)[0][0]
        tensor_padded = np.zeros((np.shape(tensor)[0], np.shape(tensor)[1] + 2*pad_val,np.shape(tensor)[2] + 2*pad_val))
        tensor_padded[:,pad_val:-pad_val,pad_val:-pad_val] = tensor
        return tensor_padded    
    
    # sliding window for image augmentation
    def sliding_window_tensor(self,tensor,window_size,stride,func):
        # grab image size, set container for results
        image_size = np.shape(tensor)[1]
        results = []
        
        # slide window over input image with given window size / stride and function
        for i in np.arange(0, image_size - window_size + 1, stride):
            for j in np.arange(0, image_size - window_size + 1, stride):
                # take a window of input tensor
                tensor_window =  tensor[:,i:i+window_size, j:j+window_size]
                
                # now process entire windowed tensor at once
                tensor_window = np.array(tensor_window)
                yo = func(tensor_window)

                # store weight
                results.append(yo)
        
        # re-shape properly
        results = np.array(results)
        results = results.swapaxes(0,1)
        if func == self.conv_function:
            results = results.swapaxes(1,2)
        return results 

    # make feature map
    def make_feature_tensor(self,tensor):
        # create feature map via convolution --> returns flattened convolution calculations
        conv_stride = 1
        feature_tensor = self.sliding_window_tensor(tensor,self.kernel_size,conv_stride,self.conv_function) 

        # re-shape convolution output ---> to square of same size as original input
        num_filters = np.shape(feature_tensor)[0]
        num_images = np.shape(feature_tensor)[1]
        square_dim = int((np.shape(feature_tensor)[2])**(0.5))
        feature_tensor = np.reshape(feature_tensor,(num_filters,num_images,square_dim,square_dim))
        
        # shove feature map through nonlinearity
        feature_tensor = self.activation(feature_tensor)

        # pool feature map --- i.e., downsample it
        pool_stride = 3
        pool_window_size = 6
        downsampled_feature_map = []
        for t in range(np.shape(feature_tensor)[0]):
            temp_tens = feature_tensor[t,:,:,:]
            d = self.sliding_window_tensor(temp_tens,pool_window_size,pool_stride,self.pool_function)
            downsampled_feature_map.append(d)
        downsampled_feature_map = np.array(downsampled_feature_map)

        # return downsampled feature map --> flattened
        return downsampled_feature_map

    # our normalization function
    def normalize(self,data,data_mean,data_std):
        normalized_data = (data - data_mean)/(data_std + 10**(-5))
        return normalized_data

    # convolution layer
    def conv_layer(self,tensor,kernels):
        #### prep input tensor #####
        # pluck out dimensions for image-tensor reshape
        num_images = np.shape(tensor)[0]
        num_kernels = np.shape(kernels)[0]
        
        # create tensor out of input images (assumed to be stacked vertically as columns)
        tensor = np.reshape(tensor,(np.shape(tensor)[0],int((np.shape(tensor)[1])**(0.5)),int( (np.shape(tensor)[1])**(0.5))),order = 'F')

        # pad tensor
        kernel = kernels[0]
        self.kernel_size = np.shape(kernel)[0]
        padded_tensor = self.pad_tensor(tensor,self.kernel_size)

        #### prep kernels - reshape into array for more effecient computation ####
        self.kernels = np.reshape(kernels,(np.shape(kernels)[0],np.shape(kernels)[1]*np.shape(kernels)[2]))
        
        #### compute convolution feature maps / downsample via pooling one map at a time over entire tensor #####
        # compute feature map for current image using current convolution kernel
        feature_tensor = self.make_feature_tensor(padded_tensor)

        feature_tensor = feature_tensor.swapaxes(0,1)
        feature_tensor = np.reshape(feature_tensor, (np.shape(feature_tensor)[0],np.shape(feature_tensor)[1]*np.shape(feature_tensor)[2]),order = 'F')
        
        return feature_tensor
    
    ##### some supervised learning capabilities #####
    def load_data(self,x,y):
        self.x = x
        self.y = y
        
    def predict(self,x,w):
        # pass input data through convolutional layer
        x_conv = self.conv_layer(x,w[0])
        
        # take inner product against output of conv layer
        value = w[1][0] + np.dot(x_conv,w[1][1:])
        return value
    
    # the softmax cost function 
    def softmax(self,w):
        cost  = np.sum(np.log(1 + np.exp((-self.y)*(self.predict(self.x,w)))))
        return cost
    
    def count(self,w):
        return 0.25*np.sum((np.sign(self.predict(self.x,w)) - self.y)**2)

In [54]:
# start timer
startTime= datetime.now() 

tensor_conv_test = tensor_conv_layer()
feature_maps_2 = tensor_conv_test.conv_layer(X,kernels)

# finish timing
timeElapsed=datetime.now()-startTime 
print('Time elpased (hh:mm:ss.ms) {}'.format(timeElapsed))

Time elpased (hh:mm:ss.ms) 0:00:00.035011


In [55]:
# start timer
startTime= datetime.now() 

tensor_conv_test = naive_conv_layer()
feature_maps_1 = tensor_conv_test.conv_layer(X,kernels)

# finish timing
timeElapsed=datetime.now()-startTime 
print('Time elpased (hh:mm:ss.ms) {}'.format(timeElapsed))

Time elpased (hh:mm:ss.ms) 0:00:01.143428


In [56]:
np.linalg.norm(feature_maps_2 - new_tensors)

0.0

In [57]:
np.linalg.norm(feature_maps_2 - feat)

7.7624475775262038e-16

In [58]:
np.linalg.norm(feature_maps_1 - feat)

2.8477974810349709

In [59]:
feat.shape

(10, 512)

In [60]:
new_tensors.shape

(10, 512)

In [61]:
feature_maps_2.shape

(10, 512)

In [62]:
feature_maps_2.shape

(10, 512)

In [63]:
class naive_version_2:
    
    # sliding window for image augmentation
    def sliding_window_image(self,image, kernel_size, stride):
        windowed_image = []
        for i in np.arange(0, np.shape(image)[0]-kernel_size+1, stride):
            for j in np.arange(0, np.shape(image)[1]-kernel_size+1, stride):
                 windowed_image.append(image[i:i+kernel_size, j:j+kernel_size].flatten())

        return np.array(windowed_image)

    # pad image with appropriate number of zeros for convolution
    def pad_image(self,image,kernel_size):
        odd_nums = np.array([int(2*n + 1) for n in range(100)])
        pad_val = np.argwhere(odd_nums == kernel_size)[0][0]
        image_padded = np.zeros((np.shape(image) + 2*pad_val))
        image_padded[pad_val:-pad_val,pad_val:-pad_val] = image
        return image_padded     
    
    # activation function
    def activation(self,window):
        a = np.maximum(0,window)
        return a
    
    def make_feature_map(self,image,kernel):
        # parameters for transform
        kernel_size = kernels[0].shape[0]
        pool_kernel_size = 6
        stride = 3
    
        # pad image with zeros
        padded_image = self.pad_image(image,kernel_size)
        
       # window image
        wind_img = sliding_window_image(padded_image,kernel_size,stride = 1)
        
        # make convolution feature map - via matrix multiplication over windowed tensor 
        feature_map = np.dot(wind_img,kernel.flatten()[:,np.newaxis])
        
        # reshape convolution feature map into array
        feature_map = np.reshape(feature_map,(np.shape(image)))

        # now shove result through nonlinear activation
        feature_map = self.activation(feature_map)

        #### now pool / downsample feature map, first window then pool on each window
        wind_featmap = sliding_window_image(feature_map,pool_kernel_size,stride = stride)

        # max pool on each collected patch
        max_pool = np.max(wind_featmap,axis = 1)

        # reshape into new tensor
        max_pool = np.reshape(max_pool, (int((np.size(max_pool))**(0.5)),int((np.size(max_pool))**(0.5))))

        return max_pool

        
    def conv_layer(self,images,kernels):
        #### create image tensor from input images
        image_tensor = np.reshape(images,(np.shape(images)[0],int((np.shape(images)[1])**(0.5)),int( (np.shape(images)[1])**(0.5))),order = 'F')

        #### loop over each image, shove through filters and make feature maps, then downsample and pool
        new_tensors = []

        #### loop over images
        for image in image_tensor:
            #### loop over kernels and construct feature map for each kernel
            downsampled_feature_maps = []
            for kernel in kernels:
                downsampled_map = self.make_feature_map(image,kernel)
                downsampled_feature_maps.append(downsampled_map)
            
            ## re-shape downsampled_feature_maps and store
            new_tensors.append(downsampled_feature_maps)

        # reshape new tensor properly
        new_tensors = np.array(new_tensors)
        new_tensors = np.reshape(new_tensors, (np.shape(new_tensors)[0],np.shape(new_tensors)[1],np.shape(new_tensors)[2]*np.shape(new_tensors)[3]))
        new_tensors = np.reshape(new_tensors, (np.shape(new_tensors)[0],np.shape(new_tensors)[1]*np.shape(new_tensors)[2]),order = 'F')

        return new_tensors

In [64]:
# start timer
startTime= datetime.now() 

tensor_conv_test = naive_version_2()
feature_maps_3 = tensor_conv_test.conv_layer(X,kernels)

# finish timing
timeElapsed=datetime.now()-startTime 
print('Time elpased (hh:mm:ss.ms) {}'.format(timeElapsed))

Time elpased (hh:mm:ss.ms) 0:00:00.589309


In [65]:
np.linalg.norm(feature_maps_3 - feat)

6.6432407931151067e-16

In [70]:
class naive_version_3:
    
    # a convolution function
    def conv_function(self,window):
        conv = np.sum(self.kernel*window)
#         conv = np.dot(window.flatten(),self.kernel.flatten()[:,np.newaxis])

        return conv

    # a pooling function
    def pool_function(self,window):
        pool = np.max(window)
        return pool

    # activation function
    def activation(self,window):
        a = np.maximum(0,window)
        return a
    
    # pad image with appropriate number of zeros for convolution
    def pad_image(self,image,kernel_size):
        odd_nums = np.array([int(2*n + 1) for n in range(100)])
        pad_val = np.argwhere(odd_nums == kernel_size)[0][0]
        image_padded = np.zeros((np.shape(image) + 2*pad_val))
        image_padded[pad_val:-pad_val,pad_val:-pad_val] = image
        return image_padded          
    
    # sliding window function, convolution or pooling done on each window
    def sliding_window_image(self,image,window_size,stride,func):
        # grab image size, set container for results
        image_size = np.shape(image)[0]
        results = []

        # slide window over input image with given window size / stride and function
        for i in np.arange(0, image_size - window_size + 1, stride):
            for j in np.arange(0, image_size - window_size + 1, stride):
                # now we have a window from our image, and use the desired 'func' to process it
                window = image[i:i+window_size,j:j+window_size]

                # process using input func
                processed_window = func(window)
                results.append(processed_window)

        # array-afy results
        results = np.array(results)

        # return results in numpy array format
        return results
    
    def make_feature_map(self,image,kernel):
        # parameters for transform
        kernel_size = kernels[0].shape[0]
        pool_kernel_size = 6
        stride = 3
    
        # pad image with zeros
        padded_image = self.pad_image(image,kernel_size)
        
        # window image
        feature_map = self.sliding_window_image(padded_image,kernel_size,stride = 1,func = self.conv_function)
        
        # reshape convolution feature map into array
        feature_map = np.reshape(feature_map,(np.shape(image)))
        
        # now shove result through nonlinear activation
        feature_map = self.activation(feature_map)

        #### now pool / downsample feature map, first window then pool on each window
        max_pool = self.sliding_window_image(feature_map,pool_kernel_size,stride = stride,func = self.pool_function)

        # reshape into new tensor
        max_pool = np.reshape(max_pool, (int((np.size(max_pool))**(0.5)),int((np.size(max_pool))**(0.5))))

        return max_pool
        
    def conv_layer(self,images,kernels):
        #### create image tensor from input images
        image_tensor = np.reshape(images,(np.shape(images)[0],int((np.shape(images)[1])**(0.5)),int( (np.shape(images)[1])**(0.5))),order = 'F')

        #### loop over each image, shove through filters and make feature maps, then downsample and pool
        new_tensors = []

        #### loop over images
        for image in image_tensor:
            #### loop over kernels and construct feature map for each kernel
            downsampled_feature_maps = []
            for kernel in kernels:
                self.kernel = kernel
                downsampled_map = self.make_feature_map(image,kernel)
                downsampled_feature_maps.append(downsampled_map)
            
            ## re-shape downsampled_feature_maps and store
            new_tensors.append(downsampled_feature_maps)

        # reshape new tensor properly
        new_tensors = np.array(new_tensors)
        new_tensors = np.reshape(new_tensors, (np.shape(new_tensors)[0],np.shape(new_tensors)[1],np.shape(new_tensors)[2]*np.shape(new_tensors)[3]))
        new_tensors = np.reshape(new_tensors, (np.shape(new_tensors)[0],np.shape(new_tensors)[1]*np.shape(new_tensors)[2]),order = 'F')

        return new_tensors

In [71]:
# start timer
startTime= datetime.now() 

tensor_conv_test = naive_version_3()
feature_maps_4 = tensor_conv_test.conv_layer(X,kernels)

# finish timing
timeElapsed=datetime.now()-startTime 
print('Time elpased (hh:mm:ss.ms) {}'.format(timeElapsed))

Time elpased (hh:mm:ss.ms) 0:00:01.073273


In [72]:
np.linalg.norm(feature_maps_3 - feature_maps_4)

5.354240412997002e-16

In [74]:
np.linalg.norm(feature_maps_4 - feat)

6.6779271899841689e-16

In [75]:
np.linalg.norm(feature_maps_4 - feature_maps_2)

4.719745629613106e-16